In [ ]:
import os, sys  
from google.colab import drive  
drive.mount('/content/drive') 

pg_path = '/content/packages'  

os.symlink('/content/drive/My Drive/Colab Notebooks', pg_path)  
sys.path.insert(0, pg_path)  

Mounted at /content/drive


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
# kobert import
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
# transformer import
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/travel_plus/data/trainset.xlsx')[['태그','테마']]
data

,태그,테마
0,"'#삼척', '#MDSF', '#모두의셀프스튜디오', '#삼척사진관', '#삼척스튜...",5
1,"'#삼척맛집', '#삼척카페', '#삼척가볼만한곳', '#삼척크로플', '#삼척쏠비...",9
2,"'#삼척스케줄', '#삼척스케줄카페', '#삼척가볼만한곳', '#삼척맛집', '#삼...",9
3,"'#삼척여행', '#삼척가볼만한곳', '#죽서루', '#은행나무', '#국화꽃향기'...",2
4,"'#삼척맛집', '#삼척대게맛집', '#삼척쏠비치리조트대게맛집', '#삼척여행', ...",8
...,...,...
468,"'#군산티안나', '#군산눈썹', '#군산입술', '#군산사람', '#군산', '#...",0
469,"'#치미창가', '#타코', '#더블치즈부리또', '#많관부', '#군산배달', '...",8
470,"'#군산', '#군산여행', '#군산카페', '#군산가볼만한곳', '#군산핫플'",9
471,"'#카페틈오늘도', '#틈카페', '#틈', '#군산카페틈', '#군산', '#군산...",9


In [ ]:
data.sample(n=10)

,태그,테마
105,"'#안동월영교', '#문보트', '#월영교', '#월영정', '#안동가볼만한곳', ...",1
131,"'#서산커뮤니티', '#서산', '#서산정보', '#서산소통', '#서산맛집', '...",0
148,"'#뒷미지', '#뒷미지수변공원', '#뒷미지연꽃', '#성주연꽃', '#연꽃명소'...",1
162,"'#풍경스냅', '#풍경스타그램', '#서울', '#서울가볼만한곳', '#송파구',...",1
89,"'#맘스타그램', '#아들맘', '#아들육아', '#아들맘소통', '#육아', '#...",2
44,'#선생님오늘은살살합시다',0
88,'#가을가을🍁',2
325,"'#경주가볼만한곳', '#아이가있는집', '#경주근교가볼만한곳', '#경주아이와가볼...",7
276,"'#양주가볼만한곳찾으신다면', '#양주카페', '#양주맛집', '#옥정맛집', '#...",9
340,"'#고령은행나무숲', '#은행나무숲', '#고령여행', '#대구근교나들이', '#대...",1


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

data_list = []

# 필요한 데이터만 추가
for d in range(len(data)):
  data_list.append([data.iloc[d][0],data.iloc[d][1]])

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
a,b = 3,4

In [ ]:
print(len(dataset_train))
print(len(dataset_test))
print(dataset_train[0])
print(dataset_test[0])

354
119
["'#고성카페', '#고성카페추천', '#봉포해수욕장', '#커피고', '#고성카페커피고', '#라떼', '#아메리카노', '#카페투어', '#바다뷰카페', '#고성가볼만한곳', '#속초주부'", 9]
["'#아산필라테스', '#아산기구필라테스', '#아산시필라테스', '#배방필라테스', '#배방기구필라테스', '#천안필라테스', '#천안기구필라테스', '#필라테스이벤트', '#아산임산부', '#아산임산부필라테스', '#아산필라테스자격증'", 0]


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64 # 문장의 최대 길이
batch_size = 64
warmup_ratio = 0.1
num_epochs = 100
max_grad_norm = 1
log_interval = 200 # 확인하기 위한 학습 로그 interval
learning_rate =  5e-5 

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
# num_workers -> 성능 따라 조절

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# 파인튜닝 -> 기존에 학습되어져 있는 모델을 기반으로 아키텍쳐를 새로운목적에맞게 변형하고 이미 학습된 모델 weights로 부터 학습을 업데이트 하는 방법
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=10,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-39-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.03400822728872299 train acc 0.984375
epoch 1 train acc 0.9820772058823529


<ipython-input-39-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/2 [00:00<?, ?it/s]

...

epoch 99 test acc 0.6410511363636364


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 100 batch id 1 loss 0.018582483753561974 train acc 1.0
epoch 100 train acc 0.9846813725490197


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 100 test acc 0.6345170454545455


**모델 저장하기**

In [ ]:
#구글드라이브 연동(여기서부터 시작할때)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/travel_plus/models/')
os.getcwd()

'/content/drive/MyDrive/travel_plus/models'

In [ ]:
path = '/content/drive/MyDrive/travel_plus/models/'
torch.save(model, path + 'travel_plus.pt')  # 전체 모델 저장

In [ ]:
torch.save(model.state_dict(), 'travel_plus_state_dict.pt')  # 모델 객체의 state_dict 저장

In [ ]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'travel_plus_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

Test

In [ ]:
import os
import torch

os.chdir('/content/drive/MyDrive/travel_plus/models/')

model1 = torch.load('travel_plus.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('travel_plus_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('travel_plus_all.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/travel_plus/models/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("스팸")
            elif np.argmax(logits) == 1:
                test_eval.append("가볼만한곳")
            elif np.argmax(logits) == 2:
                test_eval.append("가족여행")
            elif np.argmax(logits) == 3:
                test_eval.append("우정여행")
            elif np.argmax(logits) == 4:
                test_eval.append("전통")
            elif np.argmax(logits) == 5:
                test_eval.append("체험")
            elif np.argmax(logits) == 6:
                test_eval.append("캠핑")
            elif np.argmax(logits) == 7:
                test_eval.append("관람")
            elif np.argmax(logits) == 8:
                test_eval.append("맛집")
            elif np.argmax(logits) == 9:
                test_eval.append("카페")

        print(test_eval[0] + " 로 예측됩니다.")

In [ ]:
for data, answer in dataset_test:
  print('해시태그: '+data)
  predict(data)
  an = str(answer)
  print('정답: '+ an.replace('0','스팸').replace('1','가볼만한곳').replace('2','가족여행').replace('3','우정여행').replace('4','전통').replace('5','체험').replace('6','캠핑').replace('7','관람').replace('8','맛집').replace('9','카페'))
  print('---------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

해시태그: '#아산필라테스', '#아산기구필라테스', '#아산시필라테스', '#배방필라테스', '#배방기구필라테스', '#천안필라테스', '#천안기구필라테스', '#필라테스이벤트', '#아산임산부', '#아산임산부필라테스', '#아산필라테스자격증'
스팸 로 예측됩니다.
정답: 스팸
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
해시태그: '#리프패럿부천옥길점', '#부천앵무새분양', '#부천앵무새', '#체험', '#체험카페', '#이색카페', '#이색동물', '#분양', '#앵무새분양', '#부천옥길', '#부천', '#옥길동', '#앵무새', '#앵무새카페', '#부천가볼만한곳', '#광명가볼만한곳', '#시흥가볼만한곳', '#옥길동맛집', '#옥길동카페', '#부천맘', '#스타필드부천', '#옥길맘', '#항동맘', '#범박맘', '#광명맘', '#시흥맘', '#오류동맘', '#구로맘', '#부천시청맘', '#상동맘'
스팸 로 예측됩니다.
정답: 스팸
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
해시태그: '#다혜의서산', '#다혜의카페투어', '#서산카페', '#토리베이크'
카페 로 예측됩니다.
정답: 카페
---------------------------------------